In [ ]:
%%capture
%pip install einops pytorch_lightning diffusers==0.12.1 kornia librosa accelerate torchvision pandas==1.5.2

In [ ]:
import torch
import pytorch_lightning as pl

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from src import *

mpl.rcParams['figure.figsize'] = (8, 8)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Creating datasets


In [ ]:
from dataset import SpectrogramDataset

train_ds=SpectrogramDataset(target_dir='datasets/randomMIDI/PianoViolin11025/WAV/train/ins3',
                            condition_dir='datasets/randomMIDI/PianoViolin11025/WAV/train/mix',
                            return_pair=True,
                            out_channels=1
                     )


valid_ds=SpectrogramDataset(target_dir='datasets/randomMIDI/PianoViolin11025/WAV/val/ins3',
                          condition_dir='datasets/randomMIDI/PianoViolin11025/WAV/val/mix',
                          return_pair=True,
                          out_channels=1
                     )

test_ds=SpectrogramDataset(target_dir='datasets/randomMIDI/PianoViolin11025/WAV/test/ins3',
                           condition_dir='datasets/randomMIDI/PianoViolin11025/WAV/test/mix',
                           return_pair=True,
                           out_channels=1
                     )

img1,img2=train_ds[0]
print(img1.shape)

plt.subplot(1,2,1)
plt.imshow(img1.permute(1,2,0))
plt.subplot(1,2,2)
plt.imshow(img2.permute(1,2,0))

### Model setup

In [ ]:
model=PixelDiffusionConditional(train_ds,
                                valid_dataset=valid_ds,
                                lr=1e-4,
                                batch_size=4,
                                warm_up_steps=10000,
                                num_timesteps=1000,
                                loss_fn=torch.nn.functional.l1_loss)

### Training

In [ ]:
trainer = pl.Trainer(
    default_root_dir="trained_models/diffusion/L1/",
    max_epochs=3000,
    callbacks=[EMA(0.9999)],
    accelerator='auto'
)

In [ ]:
trainer.fit(model)

### Load checkpoint

In [ ]:
trainer.fit(model, ckpt_path='trained_models/diffusion/3_channels_cosine/lightning_logs/version_3/checkpoints/epoch=1104-step=138125.ckpt')

### Create Sample

In [ ]:
song_num = 3
condition,target=test_ds[song_num]
batch_input=torch.stack(1*[condition],0)

out=model(batch_input, verbose=True)

In [ ]:
plt.subplot(1,2+len(out),1)
plt.imshow(condition.permute(1,2,0))
plt.title('Input')
plt.axis('off')
for idx in range(out.shape[0]):
    plt.subplot(1,2+len(out),idx+2)
    plt.imshow(out[idx].detach().cpu().permute(1,2,0))
    plt.axis('off')
plt.subplot(1,2+len(out),2+len(out))
plt.imshow(target.permute(1,2,0))
plt.title('Ground Truth')
plt.axis('off')

In [ ]:
from torchvision.utils import save_image
phase = test_ds.get_phase(song_num)

name = test_ds.files[song_num]
test_ds.save_audio(out[0].detach().cpu(), phase, name = 'produced_' +  name)
test_ds.save_audio(target, phase, name = 'target_' + name)
test_ds.save_audio(condition, phase, name = 'condition_' + name)
save_image(out[0], 'results/produced_' + name.replace('wav','png'))
save_image(condition, 'results/condition_' + name.replace('wav','png'))
save_image(target, 'results/target_' + name.replace('wav','png'))